In [1]:
import os

In [2]:
%pwd

'/media/prajwal/HDV1/Python_Files/DL_AI/AI_Project/project-dl-main/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/media/prajwal/HDV1/Python_Files/DL_AI/AI_Project/project-dl-main'

In [5]:
# mlflow >> use to track experiments

In [6]:
# https://dagshub.com/prajwalpatil2204/project-dl-main.mlflow

In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/prajwalpatil2204/project-dl-main.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="prajwalpatil2204"
os.environ["MLFLOW_TRACKING_PASSWORD"]="93c20ae46965bedc7ee24237d553b393d8dc7523"

In [8]:
import mlflow
import os

mlflow.set_tracking_uri("https://dagshub.com/prajwalpatil2204/project-dl-main.mlflow")

os.environ["MLFLOW_TRACKING_USERNAME"]="prajwalpatil2204"
os.environ["MLFLOW_TRACKING_PASSWORD"]="93c20ae46965bedc7ee24237d553b393d8dc7523"

In [9]:
import tensorflow as tf

2025-04-14 20:40:01.663140: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-14 20:40:01.672243: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-14 20:40:01.800341: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 20:40:04.369566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-04-14 20:40:04.369566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
# entity

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
# confuguration manager 

from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/prajwalpatil2204/project-dl-main.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
# components

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model", keras_model_kwargs={"save_format": "h5"})
            else:
                mlflow.keras.log_model(self.model, "model", keras_model_kwargs={"save_format": "h5"})

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


/home/prajwal/miniconda3/envs/dlenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-04-14 20:40:12.904619: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-04-14 20:40:12.904619: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-04-14 20:40:13.271244: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-04-14 20:40:13.271244: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


1/9 ━━━━━━━━━━━━━━━━━━━━ 1:39 12s/step - accuracy: 1.0000 - loss: 0.0496

2025-04-14 20:40:23.485246: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-04-14 20:40:23.894824: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-04-14 20:40:23.894824: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


2/9 ━━━━━━━━━━━━━━━━━━━━ 1:19 11s/step - accuracy: 1.0000 - loss: 0.0390

2025-04-14 20:40:34.881188: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


9/9 ━━━━━━━━━━━━━━━━━━━━ 94s 10s/step - accuracy: 1.0000 - loss: 0.0280
9/9 ━━━━━━━━━━━━━━━━━━━━ 94s 10s/step - accuracy: 1.0000 - loss: 0.0280


2025/04/14 20:41:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/04/14 20:42:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpp9i6xkfq/model, flavor: tensorflow), fall back to return ['tensorflow==2.16.1']. Set logging level to DEBUG to see the full traceback.
2025/04/14 20:42:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpp9i6xkfq/model, flavor: tensorflow), fall back to return ['tensorflow==2.16.1']. Set logging level to DEBUG to see the full traceback.
/home/prajwal/miniconda3/envs/dlenv/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported 